# Mood Classification using CNN (HAPPY | SAD)

## Step 1: Import necessary libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Step 2: Prepare the data
We will use ImageDataGenerator to load and augment the images

In [2]:
# Directories for the datasets
train_dir = r'C:\Users\SAMEER\Projects\Deep Learning\CNN\Mood Classification using CNN\training'
validation_dir = r'C:\Users\SAMEER\Projects\Deep Learning\CNN\Mood Classification using CNN\validation'
test_dir = r'C:\Users\SAMEER\Projects\Deep Learning\CNN\Mood Classification using CNN\testing'

# Image data generators with data augmentation for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate training dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=3,
    class_mode='binary'
)

# Load and iterate validation dataset
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(200, 200),
    batch_size=3,
    class_mode='binary'
)

Found 12 images belonging to 2 classes.
Found 5 images belonging to 2 classes.


## Step 3: Build the model
We will build a simple CNN model.

In [3]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# COMPILE
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

## Step 4: Train the model
We will use EarlyStopping and ModelCheckpoint callbacks to prevent overfitting and save the best model.

In [4]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
#    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/20


4/4 [==============================] - 3s 373ms/step - loss: 0.6918 - accuracy: 0.7500 - val_loss: 1.1009 - val_accuracy: 0.6667
Epoch 2/20
4/4 [==============================] - 1s 272ms/step - loss: 1.4746 - accuracy: 0.4167 - val_loss: 0.6498 - val_accuracy: 0.6667
Epoch 3/20
4/4 [==============================] - 1s 265ms/step - loss: 0.6065 - accuracy: 0.6667 - val_loss: 0.6858 - val_accuracy: 0.6667
Epoch 4/20
4/4 [==============================] - 1s 235ms/step - loss: 0.6699 - accuracy: 0.5833 - val_loss: 0.6971 - val_accuracy: 0.3333
Epoch 5/20
4/4 [==============================] - 1s 253ms/step - loss: 0.6912 - accuracy: 0.5833 - val_loss: 0.5576 - val_accuracy: 1.0000
Epoch 6/20
4/4 [==============================] - 1s 257ms/step - loss: 0.6525 - accuracy: 0.5833 - val_loss: 0.6172 - val_accuracy: 0.6667
Epoch 7/20
4/4 [==============================] - 1s 230ms/step - loss: 0.6970 - accuracy: 0.5833 - val_loss: 0.6936 - val_accuracy: 0.6667
Epoch 8/20
4/4 [==

## Step 5: Evaluate the model
We will evaluate the model on the test set. 

In [5]:
# Evaluate on validation data
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Test accuracy: {test_accuracy:.2f}')

1/1 [==============================] - 0s 117ms/step - loss: 0.3370 - accuracy: 1.0000
Test accuracy: 1.00


In [6]:
# Save the entire model to a HDF5 file
model.save('mood_model.h5')

C:\Users\SAMEER\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('mood_model.h5')

## Step 6: Make predictions
Finally, we will make predictions on new images. 

In [8]:
# Function to classify an image
def classify_image(img_path):
    img = load_img(img_path, target_size=(200, 200))  # Use (200, 200) to match the model's input size
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = loaded_model.predict(img_array)
    if prediction[0] < 0.5:
        print("Happy")
    else:
        print("Sad")
        
# Example usage
img_path = r'C:\Users\SAMEER\Projects\Deep Learning\CNN\Mood Classification using CNN\testing\happy\happy2.jpg'
classify_image(img_path)

1/1 [==============================] - 0s 130ms/step
Happy


In [9]:
# Example usage
img_path = r'C:\Users\SAMEER\Projects\Deep Learning\CNN\Mood Classification using CNN\training\sad\s1.jpeg'
classify_image(img_path)

1/1 [==============================] - 0s 27ms/step
Sad


### Model is working well and predicting correct mood happy and sad with provided images 